<a href="https://colab.research.google.com/github/hexagramg/iu5-BI/blob/master/lab/lab2_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/google-research/tf-slim

  Cloning https://github.com/google-research/tf-slim to /tmp/pip-req-build-le5_kkdj
  Running command git clone -q https://github.com/google-research/tf-slim /tmp/pip-req-build-le5_kkdj
  Created wheel for tf-slim: filename=tf_slim-1.1-cp36-none-any.whl size=357897 sha256=e05d7352dd604ff4a7478abbc855f31e1bced1a3a6e05c4483a42c5d5e0224f1
  Stored in directory: /tmp/pip-ephem-wheel-cache-rsh9pk8k/wheels/bb/98/dc/eba6500d756d16f6ff371b39ed733d26cec1b0b0085e1cb0df
Successfully built tf-slim


In [2]:
import tensorflow.compat.v1 as tf
import tf_slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


Загрузим необходимое по условию окружение:

In [0]:
env = gym.make('CartPole-v0')

Напишем функцию, подсчитывающую награду:

In [0]:
gamma = 0.99

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

Определим класс агента, который будет работать в описанной выше среде.

In [0]:
class agent():
  def __init__(self, lr, s_size, a_size, h_size):
    self.state_in= tf.placeholder(shape=[None, s_size], dtype=tf.float32)
    hidden = slim.fully_connected(self.state_in, h_size, biases_initializer=None, activation_fn=tf.nn.relu)
    self.output = slim.fully_connected(hidden, a_size, activation_fn=tf.nn.softmax, biases_initializer=None)
    self.chosen_action = tf.argmax(self.output, 1)

    self.reward_holder = tf.placeholder(shape=[None], dtype=tf.float32)
    self.action_holder = tf.placeholder(shape=[None], dtype=tf.int32)

    self.indexes = tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output)[1] + self.action_holder
    self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)

    self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs) * self.reward_holder)

    tvars = tf.trainable_variables()
    self.gradient_holders = []
    for idx,var in enumerate(tvars):
      placeholder = tf.placeholder(tf.float32,name=str(idx) + "_holder")
      self.gradient_holders.append(placeholder)

    self.gradients = tf.gradients(self.loss, tvars)

    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, tvars))

Обучаем агента:

In [11]:
tf.reset_default_graph()

myAgent = agent(lr=1e-2, s_size=4, a_size=2, h_size=8)

total_episodes = 5000
max_ep = 500
update_frequency = 5

init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  i = 0
  total_reward = []
  total_length = []
      
  gradBuffer = sess.run(tf.trainable_variables())
  for ix,grad in enumerate(gradBuffer):
    gradBuffer[ix] = grad * 0

  while i < total_episodes:
    s = env.reset()
    running_reward = 0
    ep_history = []
    for j in range(max_ep):
      a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s]})
      a = np.random.choice(a_dist[0],p=a_dist[0])
      a = np.argmax(a_dist == a)

      s1, r, done, _ = env.step(a)
      ep_history.append([s, a, r, s1])
      s = s1
      running_reward += r
      if done == True:
        ep_history = np.array(ep_history)
        ep_history[:,2] = discount_rewards(ep_history[:,2])
        feed_dict = {
            myAgent.reward_holder: ep_history[:,2],
            myAgent.action_holder: ep_history[:,1],
            myAgent.state_in:      np.vstack(ep_history[:,0])
          }
        grads = sess.run(myAgent.gradients, feed_dict=feed_dict)
        for idx,grad in enumerate(grads):
          gradBuffer[idx] += grad

        if i % update_frequency == 0 and i != 0:
          feed_dict= dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
          _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
          for ix,grad in enumerate(gradBuffer):
            gradBuffer[ix] = grad * 0

        total_reward.append(running_reward)
        total_length.append(j)
        break

    if i % 100 == 0:
      print(f"Эпизод {i} средняя награда {np.mean(total_reward[-100:])}, осталось {total_episodes - i}")
    i += 1

Эпизод 0 средняя награда 19.0, осталось 5000
Эпизод 100 средняя награда 23.85, осталось 4900
Эпизод 200 средняя награда 33.95, осталось 4800
Эпизод 300 средняя награда 45.64, осталось 4700
Эпизод 400 средняя награда 52.17, осталось 4600
Эпизод 500 средняя награда 56.59, осталось 4500
Эпизод 600 средняя награда 64.16, осталось 4400
Эпизод 700 средняя награда 95.82, осталось 4300
Эпизод 800 средняя награда 124.5, осталось 4200
Эпизод 900 средняя награда 142.51, осталось 4100
Эпизод 1000 средняя награда 149.13, осталось 4000
Эпизод 1100 средняя награда 137.28, осталось 3900
Эпизод 1200 средняя награда 147.8, осталось 3800
Эпизод 1300 средняя награда 175.09, осталось 3700
Эпизод 1400 средняя награда 191.64, осталось 3600
Эпизод 1500 средняя награда 199.07, осталось 3500
Эпизод 1600 средняя награда 198.72, осталось 3400
Эпизод 1700 средняя награда 199.75, осталось 3300
Эпизод 1800 средняя награда 193.42, осталось 3200
Эпизод 1900 средняя награда 194.3, осталось 3100
Эпизод 2000 средняя нагр